# AIN501 - Deep learning
## Assignment 1 - CIFAR 10
### Tran Trong Hieu - MSE23185

#### I. Import and check GPU

In [1]:
# Import section
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


#### II. Load dataset CIFAR-10

In [2]:
# Transform
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Batch size
batch_size_train = 64
batch_size_test = 100

# Load dataset CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size_train, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size_test, shuffle=False, num_workers=2)

# Define 10 classes of CIFAR-10 dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:03<00:00, 48.6MB/s]


#### III. Build model with informations:
1. Using CNN with ELU activation function
2. Loss fuction: Cross entropy loss
3. Optimizer: Adam
4. Early stop and check point

In [3]:
# CNN model with ELU activation fuction
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.conv1 = nn.Sequential(
      nn.Conv2d(3, 64, 3, padding=1),
      nn.BatchNorm2d(64),
      nn.ELU(),
      nn.Conv2d(64, 64, 3, padding=1),
      nn.BatchNorm2d(64),
      nn.ELU(),
      nn.MaxPool2d(2),
      nn.Dropout(0.3)
    )

    self.conv2 = nn.Sequential(
      nn.Conv2d(64, 128, 3, padding=1),
      nn.BatchNorm2d(128),
      nn.ELU(),
      nn.Conv2d(128, 128, 3, padding=1),
      nn.BatchNorm2d(128),
      nn.ELU(),
      nn.MaxPool2d(2),
      nn.Dropout(0.4)
    )

    self.conv3 = nn.Sequential(
      nn.Conv2d(128, 256, 3, padding=1),
      nn.BatchNorm2d(256),
      nn.ELU(),
      nn.Conv2d(256, 256, 3, padding=1),
      nn.BatchNorm2d(256),
      nn.ELU(),
      nn.MaxPool2d(2),
      nn.Dropout(0.5)
    )

    self.fc = nn.Sequential(
      nn.Flatten(),
      nn.Linear(256 * 4 * 4, 512),
      nn.ELU(),
      nn.Dropout(0.5),
      nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.fc(x)
    return x

model = CNN().to(device)

#### IV. Define optimizer, loss function and early stop

In [5]:
# Define optimizer, loss function and early stop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

best_loss = float('inf')
early_stop_count = 0
patience = 7
save_path = 'cnn_model.pth'

#### V. Train model

In [6]:
# Training
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
      images, labels = images.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(trainloader.dataset)
    print(f"Epoch [{epoch+1}/100], Loss: {epoch_loss:.4f}")
    scheduler.step(epoch_loss)

    # Early stopping logic
    if epoch_loss < best_loss:
      best_loss = epoch_loss
      early_stop_count = 0
      torch.save(model.state_dict(), save_path)
      print("Saved Best Model")
    else:
      early_stop_count += 1
      if early_stop_count >= patience:
          print("Early stopping triggered")
          break

# Load best model
model.load_state_dict(torch.load(save_path))

Epoch [1/100], Loss: 1.5513
Saved Best Model
Epoch [2/100], Loss: 1.1103
Saved Best Model
Epoch [3/100], Loss: 0.9459
Saved Best Model
Epoch [4/100], Loss: 0.8581
Saved Best Model
Epoch [5/100], Loss: 0.8001
Saved Best Model
Epoch [6/100], Loss: 0.7457
Saved Best Model
Epoch [7/100], Loss: 0.7039
Saved Best Model
Epoch [8/100], Loss: 0.6673
Saved Best Model
Epoch [9/100], Loss: 0.6225
Saved Best Model
Epoch [10/100], Loss: 0.5979
Saved Best Model
Epoch [11/100], Loss: 0.5633
Saved Best Model
Epoch [12/100], Loss: 0.5454
Saved Best Model
Epoch [13/100], Loss: 0.5212
Saved Best Model
Epoch [14/100], Loss: 0.5010
Saved Best Model
Epoch [15/100], Loss: 0.4818
Saved Best Model
Epoch [16/100], Loss: 0.4548
Saved Best Model
Epoch [17/100], Loss: 0.4524
Saved Best Model
Epoch [18/100], Loss: 0.4324
Saved Best Model
Epoch [19/100], Loss: 0.4164
Saved Best Model
Epoch [20/100], Loss: 0.4050
Saved Best Model
Epoch [21/100], Loss: 0.3931
Saved Best Model
Epoch [22/100], Loss: 0.3828
Saved Best Mod

<All keys matched successfully>

#### VI. Evaluation model

In [7]:
# Evaluation
def evaluate(model, testloader):
    model.eval()
    total = 0
    correct = 0
    class_correct = [0] * 10
    class_total = [0] * 10
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                if label == pred:
                    class_correct[label] += 1
                class_total[label] += 1

    print(f"\nTest Loss: {total_loss / total:.6f}\n")
    for i in range(10):
        acc = 100 * class_correct[i] / class_total[i]
        print(f"Test Accuracy of {classes[i]:10}: {acc:.0f}% ({class_correct[i]}/{class_total[i]})")
    print(f"Test Accuracy (Overall): {100 * correct / total:.0f}% ({correct}/{total})")

evaluate(model, testloader)


Test Loss: 0.462660

Test Accuracy of plane     : 90% (901/1000)
Test Accuracy of car       : 95% (950/1000)
Test Accuracy of bird      : 85% (852/1000)
Test Accuracy of cat       : 80% (796/1000)
Test Accuracy of deer      : 91% (909/1000)
Test Accuracy of dog       : 83% (831/1000)
Test Accuracy of frog      : 94% (939/1000)
Test Accuracy of horse     : 90% (905/1000)
Test Accuracy of ship      : 94% (943/1000)
Test Accuracy of truck     : 94% (942/1000)
Test Accuracy (Overall): 90% (8968/10000)
